# DPA3: A Graph Neural Network for the Era of Large Atomistic Models

# 1. Introduction

***DPA3*** is a multi-layer graph neural network founded on line graph series (LiGS), designed explicitly for the era of [large atomistic models (LAMs)](https://www.aissquare.com/openlam). The generalization error of the DPA3 model adheres to the scaling law. The scalability in the number of model parameters is attained by stacking additional layers within DPA3. Additionally, the model employs a dataset encoding mechanism that decouples the scaling of training data size from the model size within its multi-task training framework.

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/8ca7545d-d0f6-4952-9d6a-fc615a950f9e.png" alt="" width="600" style="display: block; margin: auto;">

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/c5adcd1b-0721-400b-a7c9-c8e4f25a78ec.png" alt="" width="600" style="display: block; margin: auto;">

When trained as problem-oriented potential energy models, the DPA3 model exhibits superior accuracy in the majority of benchmark cases, encompassing systems with diverse features, including molecules, bulk materials, surface and cluster catalysts, two-dimensional materials, and battery materials. When trained as a LAM on the OpenLAM-v1 dataset, the DPA-3.1-3M model exhibits state-of-the-art performance in the LAMBench benchmark suite for LAMs, demonstrating lowest overall zero-shot generalization error across 17 downstream tasks from a broad spectrum of research domains. This performance suggests superior accuracy as an out-of-the-box potential model, requiring minimal fine-tuning data for downstream scientific applications.

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/79ee197d-4283-4403-b3fe-5b0758e992a8.png" alt="" width="600" style="display: block; margin: auto;">



<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/0ab27a7f-a35b-463c-8606-e740a8d0d328.png" alt="" width="600" style="display: block; margin: auto;">


Reference: 

[1] Zhang, D., Peng, A., Cai, C., Li, W., Zhou, Y., Zeng, J., ... & Wang, H. (2025). Graph neural network model for the era of large atomistic models. [arXiv preprint arXiv:2506.01686](https://arxiv.org/abs/2506.01686).


# 2. 论文阅读

## 2.1 摘要 (Abstract)

本文提出了DPA-3，一个基于“线图系列”（Line Graph Series, LiGS）的多层图神经网络，专为LAMs时代设计。

- 目标：大型原子模型（LAMs）旨在普适性地表示由密度泛函理论（DFT）定义的原子系统基态势能面。

- 核心理论：“缩放定律”（Scaling law）至关重要，它指出模型的泛化能力会随着模型尺寸、训练数据和计算资源的增加而持续提升。

- 关键特性：

    1. 遵循缩放定律：DPA-3的泛化误差遵循缩放定律 。通过堆叠更多层数，可以扩展模型参数量。


    2. 多任务学习：模型采用“数据集编码”机制，在多任务训练框架中，将训练数据规模的扩展与模型大小的扩展解耦。

- 性能表现：

    1. 作为专用势能：当作为面向特定问题的势能模型时，DPA-3在包含分子、块状材料、催化剂、二维材料和电池材料等多种系统的基准测试中，表现优于大多数现有模型。

    2. 作为通用模型（LAM）：在OpenLAM-v1数据集上训练后，DPA-3.1-3M模型在LAMBench基准测试套件中展现了最先进的性能，在17个下游任务中实现了最低的总体零样本泛化误差 。这表明它作为一个“开箱即用”的势能模型具有极高的精度，或仅需极少的微调数据。

## 2.2 引言 (Introduction)

- 物理背景：原子系统的基础是薛定谔方程，其基态解定义了一个普适的势能面（PES）。在实际应用中，DFT是计算PES的可行近似方法，但其计算复杂度随电子数成立方增长，成本高昂。

- MLIPs的兴起：机器学习原子间势（MLIPs）作为DFT的代理模型出现，将计算成本降至线性尺度，同时保持了相当的精度。但传统MLIPs通常是为特定问题定制的，在新体系上需要重新训练。

- LAMs的出现与挑战：这一需求催生了旨在普适性表示PES的LAMs。其可行性基于DFT解的普适性。LAMs的目标是成为“开箱即用”的势能或只需少量数据即可微调的模型。尽管已有成功应用（如GNoME模型发现了38.1万个新稳定结构），但现有LAMs的泛化能力与专用MLIPs仍有差距。

- 缩放定律的重要性：提升LAMs泛化能力的关键在于系统性地扩大数据集、模型参数和计算资源，即遵循“缩放定律”。然而，对LAMs中缩放定律的探索还很有限，并且基于GNN的架构可能因“过平滑”效应而面临挑战。

- 数据与训练的挑战：由于不同研究中DFT计算设置（如交换相关泛函、基组）不一致，直接合并数据训练LAMs很困难。一种方法是扩展数据集（如Open Materials 2024），但这难以满足所有领域的需求。另一种方法是多任务训练，但传统方法需要为每个数据集增加一个拟合头，导致模型随数据集数量增加而膨胀，可扩展性差。

- 物理约束：LAMs必须遵守物理定律，即模型应是光滑且保守的（能量守恒），并满足平移、旋转和置换不变性。

本文提出的DPA-3模型旨在解决以上所有问题：它基于LiGS构建，被设计为遵循缩放定律；它采用数据集编码实现可扩展的多任务训练；并且严格遵守所有相关的物理定律。

## 2.3 结果 (Results)

### 2.3.1 DPA-3: 基于线图系列的图神经网络

> 线图变换 (Line Graph Transform)：

给定一个图G，线图变换L(G)会生成一个新图。这个过程分两步：

1. G中的每条边成为L(G)中的一个顶点；

2. 如果G中的两条边共享一个共同的顶点，那么它们在L(G)中对应的两个顶点之间就形成一条边。

> 线图系列 (LiGS, Line Graph Series)：

从一个初始图$G^{(1)}$开始，可以递归地生成一系列图$\{G^{(1)}, G^{(2)}, ..., G^{(K)}\}$，其中$G^{(k)}=L(G^{(k-1)})$，称这个图$G^{(k)}$的阶数为k。

> 与原子系统相对应：

在原子系统中，$G^{(1)}$的顶点是原子，边是邻近的原子对。每个原子的近邻原子由在距离其一个自定义的截断长度$r_c$范围内的原子组成。

通过LiGS，二阶图$G^{(2)}$的顶点对应于“键”，三阶图$G^{(3)}$的顶点对应于同一个原子与另外两个近邻原子构成的化学键的“角”，四阶图$G^{(4)}$的顶点则对应于四个原子形成的“二面角”。

<img src="https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/d1789633d10145e7aebd3f76ec3f71d4/8ca7545d-d0f6-4952-9d6a-fc615a950f9e.png" alt="" width="600" style="display: block; margin: auto;">

***模型架构*** (上图)

- 信息表示：DPA-3是在LiGS上进行信息传递的多层神经网络。每个图$G^{(k)}$的第l层的顶点和边的特征分别用$v_{\alpha}^{(k,l)}$和$e_{\alpha\beta}^{(k,l)}$来表示，其维度分别为$d_v$和$d_e$。

- 特征信息更新：更新过程是迭代和递归的
    1. 顶点的信息通过其邻边信息进行卷积来更新
    2. 边特征则基于自身和两端顶点的特征来更新
    3. 更新采用残差形式以保证稳定性

- 信息共享：一个关键设计是，$G^{(k)}$的顶点特征等同于$G^{(k-1)}$的边特征。因此，无需为$k>1$的图存储顶点特征，其更新直接传递给$G^{(k-1)}$作为边的特征。

- 最终描述子：最终只有初始图$G^{(1)}$的顶点特征被用作描述原子局部环境的描述子。

- 多任务与能量计算：
    1. 在多任务训练中，描述子会与一个数据集编码（one-hot向量）拼接。
    2. 增强后的特征被送入一个拟合MLP来预测原子能量，然后求和得到总能量。
    3. 力与 virial 张量通过对能量进行反向传播得到，这保证了模型的内在保守性。

设计选择：理论上可以将所有图的最终顶点特征通过某种池化（pooling）方式聚合起来 。

性能表现：研究发现，这种聚合方法在浅层网络中能提升性能，但在深层网络中反而会导致性能下降。

最终方案：考虑到模型的性能主要通过增加网络深度（L）来提升，因此作者选择只使用$G^{(1)}$的最终顶点特征来构建描述子，因为这在深度模型中效果最好。

同时，由于$G^{(1)}$的顶点直接对应系统中的原子，使用它的顶点特征作为原子描述子也是最自然的选择。

故：描述子来自初始图$G^{(1)}$，是因为我们需要一个以“原子”为单位的描述子。但这个描述子是在经历了L层复杂的、包含了所有高阶图信息的更新之后才形成的最终产物，因此它非常强大和富有表现力。

### 2.3.2 基准测试 (Benchmarking)

* ***SPICE-MACE-OFF 数据集 (图2a)***：这是一个有机小分子数据集。
    * DPA-3-L6（小模型）用20%的参数量（1.3M vs 6.9M），实现了比MACE(L)低34%的能量误差（LWAMAE），速度快3.7倍。
    * DPA-3-L24（大模型）用少30%的参数量，实现了比MACE(L)低66%的能量误差。
    * 模型性能随层数增加而提升。eSEN模型在该测试中表现最佳。
* ***TorsionNet-500 数据集 (图2b)***：用于评估扭转势能面的精度。
    * DPA-3-L24在所有指标上均优于其他模型，扭转势垒高度MAE比MACE(L)低60%。
    * DPA-3-L12和L24的$NABH_{h}$指标为0，意味着所有扭转势垒预测误差都在1 kcal/mol以内。
* ***水/冰体系 (图2c)***：评估模型在凝聚态体系的表现。
    * 使用0.1%的训练数据，DPA-3-L12的总体RMSE低于NequIP。
    * 在此案例中，缩放定律不明显，DPA-3-L24性能反而不如L12，可能是因为数据集多样性有限导致大模型过拟合。
* ***AIMD材料数据集 (图2d)***：包含催化、二维材料和多孔材料（沸石）三个数据集。
    * DPA-3模型随着尺寸增大，精度稳步提升。DPA-3-L24在沸石数据集上的精度略低于AlphaNet。
* ***DPA2测试集 (图2e)***：这是一个包含18个不同领域数据集的综合测试集。
    * DPA-3-L24在能量和力的LWARMSE指标上均表现最佳。
* ***LiGS阶数K的影响 (图3)***：
    * 结果显示，将阶数从K=1增加到K=2能显著提升精度，但继续增加到K=3反而导致精度下降。
    * 原因可能是K=2的角信息已足够，而K=3的二面角项引入了不必要的训练复杂性。因此，K=2被定为默认的最优配置。


### 2.3.3 缩放定律 (Scaling Law)

* ***Matbench Discovery Leaderboard (表1)***：这是一个评估MLIPs预测无机晶体稳定性的基准。DPA-3-L24在2025年5月27日的所有合规模型中，取得了第二好的综合性能得分（CPS）。
* ***实验验证***：使用MPtrj数据集进行训练，在WBM测试集上评估泛化误差。通过改变模型层数（M）、训练步数（S）和计算预算（C），分析了泛化MAE的变化。
* ***经验公式***：使用了经验幂律关系式 $MAE(M,S,C) = \alpha_{m}M^{\beta_{m}} + \alpha_{s}S^{\beta_{s}} + \alpha_{c}C^{\beta_{c}}$ 进行拟合。
* ***结果 (图4)***：拟合结果的R²值高达0.981，有力地证明了DPA-3模型确实遵循缩放定律，其性能可以通过系统性地扩展资源来提升。


### 2.3.4 大型原子模型: DPA-3.1-3M

* ***模型训练***：使用DPA-3架构在OpenLAM-v1数据集集合（包含31个不同领域的数据集）上进行多任务训练，得到了DPA-3.1-3M模型（L=16层，K=2阶，约326万参数）。
* ***LAMBench评估***：在LAMBench基准测试套件的力场任务上评估了模型的零样本（zero-shot）泛化能力。
* ***数据集编码的作用 (图5)***：
    * 评估了两种情况：一种是为不同测试任务手动选择最匹配的DFT设置（数据集编码），标记为`DPA-3.1-3M-bestXC`。另一种是统一使用MPtrj的数据集编码，标记为`DPA-3.1-3M`。
    * 结果显示，`DPA-3.1-3M`的平均误差为0.23，已是SOTA水平。而`DPA-3.1-3M-bestXC`通过利用先验知识，进一步将平均误差降至0.20，表现出最佳性能。
* ***与单任务模型的比较***：单任务训练的Orb-v3模型表现也很有竞争力（平均误差0.24），但其参数量远大于DPA-3.1-3M（25.5M vs 3.26M）。
* ***多任务方案的优越性***：论文对比了DPA-3使用的统一数据集编码方案和传统为每个任务设置独立拟合网络的方案。结果表明，统一编码方案在性能相当或更好的情况下，参数量大幅减少（3.26M vs 7.69M），证明了其作为可扩展LAM开发方案的优势。


## 2.4 讨论 (Discussion)

* ***总结***：DPA-3模型架构满足了LAMs的需求：遵循物理原理、展现缩放定律、在多任务训练中参数规模恒定。
* ***未来方向***：提升DPA-3泛化能力的路径是明确的：
  1) 通过优化实现或模型并行化来增大模型尺寸；
  2) 整合更多领域的训练数据（如OMol25数据集）来扩大数据规模。
* ***潜在局限性***：论文指出，DPA-3在某些小分子AIMD数据集上表现不如MACE等“等变”模型。未来需要进一步研究引入等变特征是否能增强其在这些体系中的泛化能力。


方法 (Methods)：

## 2.5 数据集 (Datasets)

这部分详细介绍了所有基准测试中使用的数据集来源、构成和划分方法，包括：SPICE-MACE-OFF、TorsionNet-500、液态水和冰的动力学、Cu上的甲酸盐分解、有缺陷的双层石墨烯、沸石数据集、DPA2测试集、以及Materials Project轨迹数据集 (MPtrj)。（略）

## 2.6 DPA-3模型架构 (DPA-3 model architecture)

这部分将深入阐述DPA-3模型的数学框架和实现细节。

### 2.6.1 能量、力与维里张量 (Energy, Force, and Virial Tensor)

首先，模型的基本假设是，一个包含N个原子的系统的总能量 $E$ 可以被分解为每个原子的能量贡献 $E_i$ 之和：

$$ E = \sum_i E_i $$

这是一个局域能量模型的核心思想，使得总能量的计算可以扩展到大规模体系。基于这个能量表达式，作用在原子 $i$ 上的力 $F_i$ 和体系的维里张量 $\Xi_{pq}$ 可以通过对能量求导得到：

$$ F_i = -\nabla_{r_i} E $$

$$ \Xi_{pq} = -\sum_{r}\frac{\partial E}{\partial h_{rp}}h_{rq} $$

* ***力的计算***：力是总能量对原子坐标 $r_i$ 的负梯度。由于DPA-3模型是平滑可导的，这个导数可以通过自动微分（反向传播）精确计算，这保证了模型是***能量保守的***，是进行分子动力学模拟的先决条件。
* ***维里张量的解释***：维里张量 $\Xi_{pq}$ 描述了体系内部的应力状态，它通过总能量 $E$ 对模拟盒子（晶胞）的晶格矢量 $h$ 求导得到。具体来说，$h_{rp}$ 是第 $r$ 个晶格矢量的第 $p$ 个分量。维里张量在需要控制或计算体系压强的模拟中（如在NPT或NVT系综下）至关重要。

DPA-3模型构建原子能量贡献 $E_i$ 的具体形式如下：

$$ E_{i} = \mathcal{F}(v_{i}^{(1,L)}, c(\mathcal{D}_{m})) + e_{m}(Z_{i}) $$

* $\mathcal{F}$ 是一个多层感知机（MLP），被称为***拟合网络***。
* $v_{i}^{(1,L)}$ 是原子 $i$ 的最终描述子，代表在图$G^{(1)}$上经过L层更新后的顶点特征。
* $c(\mathcal{D}_{m})$ 是在多任务学习中，用于区分不同数据集 $\mathcal{D}_m$ 的编码（通常是one-hot向量）。
* $e_{m}(Z_{i})$ 是一个与元素种类 $Z_i$ 和数据集 $\mathcal{D}_m$ 相关的***能量偏置项***。理想情况下，能量偏置 $e_{bias}$ 应取为真空状态下单个原子的能量。它通过对训练集的能量进行最小二乘拟合得到，其作用是校准不同来源（不同DFT计算设置）的数据集之间任意的能量零点，使得模型可以有效学习。
* 更准确地说，假设我们有 M 个数据帧，在第 m 帧中，我们有 $c_{mz}$ 个原子序数为 z 的原子，该帧的 DFT 标记能量表示为 $E_{m}^{*}$。 那么线性系统 $ \sum_{z}c_{mz}e_{bias}(z)=E_{m}^{*},\space m=1,...,M$ 在最小二乘意义上被求解。这里我们假设系统方程中的独立方程数量等于或小于数据帧的数量 M。


### 2.6.2 DPA-3 描述子
DPA-3 描述子从根本上基于图神经网络架构，该架构输入分别代表节点、边和角度特征的 $n_{i}^{0}$、$e_{ij}^{0}$ 和 $a_{ijk}^{0}$。 
这些特征通过 L 个 repflow 层（通常为六个）利用消息传递机制进行逐步更新： 
$$ n_{i}^{L},e_{ij}^{L},a_{ijk}^{L}=repflow\circ...\circ repflow(n_{i}^{0},e_{ij}^{0},a_{ijk}^{0}), $$

最终，更新后的节点（顶点）特征 $n_{i}^{L}$ 作为输出描述子。

在详细介绍具体的更新机制之前，我们必须澄清两种图结构的范围：
首先 repflow 层仅需要原子 i 的邻近原子的信息，即所有落在以原子 i 为中心、截断半径为 $r_{c}$ 的球内的原子。因此，我们引入符号 $N_{r_{c}}(i)$，它代表 i 的所有邻居的集合，即 $N_{r_{c}}(i)=\{j:j\ne i,|r_{j}-r_{i}|<r_{c}\}$。系统中可能的最大近邻原子数表示为 $N_{r_{c}}^{m}$，因此对任意 i 有 $|N_{r_{c}}(i)|\le N_{r_{c}}^{m}$。

对于仅涉及节点和边特征的更新，通常采用一个较大的截断半径 $r_{c}^{e}$（例如 6Å）来包含中心原子周围的原子。原子 i 的这一组邻近原子表示为 $N_{r_{c}^{e}}(i)$。相反，对于包含角度特征的更新，由于计算复杂度较高，我们选择一个较小的截断半径 $r_{c}^{a}$（例如 4 Å）。原子 i 的相应邻近原子集合称为 $N_{r_{c}^{a}}(i)$。


### 2.6.3 特征初始化

在第0层（l=0），模型的特征需要从系统的初始几何结构中进行初始化，为后续的信息传递提供基础。

* ***$G^{(1)}$顶点 (原子)***:
    $$ v_{i}^{(1,0)} = \mathrm{one\_hot}(Z_i) $$
    使用原子序数 $Z_i$ 的独热编码（one-hot encoding）作为原子的初始特征。

* ***$G^{(1)}$边 (键)***:
    $$ e_{ij}^{(1,0)} = \phi_{e}^{(1)}(r_{ij}) $$
    使用原子间的距离 $r_{ij}$ 作为输入，通过一个MLP网络 $\phi_e^{(1)}$ 嵌入后，得到键的初始特征。

* ***$G^{(2)}$边 (角)***:
    $$ e_{(ij)(im)}^{(2,0)} = \phi_{e}^{(2)}(\cos(\theta_{ijm})) $$
    使用由三原子构成的键角 $\theta_{ijm}$ 的余弦值作为输入，通过MLP网络 $\phi_e^{(2)}$ 嵌入后，得到角的初始特征。

* ***$G^{(3)}$边 (二面角)***:
    $$ e_{(ijm)(ijn)}^{(3,0)} = \phi_{e}^{(3)}(\cos(\eta_{mijn})) $$
    使用由四原子构成的二面角 $\eta_{mijn}$ 的余弦值作为输入，通过MLP网络 $\phi_e^{(3)}$ 嵌入后，得到二面角的初始特征。

这些初始化步骤确保了每一层级的图都拥有与其几何意义相匹配的初始信息。

### 2.6.4 特征更新的递归机制

模型通过一个递归的、在LiGS中所有图上应用的残差更新机制来演化特征。对于第 $l$ 层的图 $k$ 的顶点特征 $v_{\alpha}^{(k,l)}$ 和边特征 $e_{\alpha\beta}^{(k,l)}$，其更新到第 $l+1$ 层的过程如下：

首先，特征被初始化为上一层的值，然后加上一个更新量：

$$ v_{\alpha}^{(k,l+1)} \leftarrow v_{\alpha}^{(k,l+1)} + \delta_{c}^{(k,l)}u_{\alpha}^{(k,l)} $$

$$ e_{\alpha\beta}^{(k,l+1)} \leftarrow e_{\alpha\beta}^{(k,l+1)} + \delta_{s}^{(k,l)}m_{s,\alpha\beta}^{(k,l)} $$

* $\delta_c$ 和 $\delta_s$ 是可训练的步长参数，用于动态调整更新幅度，保证深度网络训练的稳定性。
* $u_{\alpha}^{(k,l)}$ 是对顶点 $\alpha$ 的核心更新量，通过对其所有邻边传递来的消息进行***卷积***（加权求和）得到。
* $m_{s,\alpha\beta}^{(k,l)}$ 是用于更新边特征的***自消息 (self-message)***。

消息的具体生成方式如下：

$$ u_{\alpha}^{(k,l)} = \phi_{u}\left(N_{m}^{-\alpha_{k}}\sum_{\beta\in\mathcal{N}^{(k)}(\alpha)}w_{\alpha\beta}^{k}m_{c,\alpha\beta}^{(k,l)}\right) $$

$$ m_{c,\alpha\beta}^{(k,l)} = \phi_{c}(v_{\alpha}^{(k,l)}, v_{\beta}^{(k,l)}, e_{\alpha\beta}^{(k,l)}) $$

$$ m_{s,\alpha\beta}^{(k,l)} = \phi_{s}(v_{\alpha}^{(k,l)}, v_{\beta}^{(k,l)}, e_{\alpha\beta}^{(k,l)}) $$

* $\phi_c$ 和 $\phi_s$ 是MLP网络，而 $\phi_u$ 可以是MLP或一个恒等映射（identity mapping）。
* $\mathcal{N}^{(k)}(\alpha)$ 表示在图 $G^{(k)}$ 中与顶点 $\alpha$ 相连的所有边的集合。
* $N_{m}^{-\alpha_{k}}$ 是一个归一化因子，其中 $N_m$ 是对最大可能邻居数的估计，而 $\alpha_k > 0$ 是一个幂次因子。

一个***至关重要***的设计细节在于：$G^{(k)}$的顶点特征 $v^{(k,l)}$ 与前一个图 $G^{(k-1)}$ 的边特征 $e^{(k-1,l)}$ 是***等同的***。基于此：

1.  模型***无需为 $k>1$ 的图维护冗余的顶点特征信息***。
2.  计算出的顶点更新量 $u_{\alpha}^{(k,l)}$ 会被***直接传递给前一个图 $G^{(k-1)}$，用于更新其边特征 $e^{(k-1,l)}$***。
3.  随后，$G^{(k-1)}$ 更新后的边特征 $e^{(k-1,l+1)}$ 就自然地成为了 $G^{(k)}$ 在新一层计算中的顶点特征 $v^{(k,l+1)}$。这个过程如图1(e)所示。



### 2.6.5 单层特征更新 ($l \rightarrow l+1$) 的详细流程

整个流程建立在一个核心事实上：***一个高阶图 $G^{(k)}$ 的顶点 (vertex)，等同于一个低阶图 $G^{(k-1)}$ 的边 (edge)***。

-   数学上表示为：$v^{(k,l)} \equiv e^{(k-1,l)}$
-   这意味着，我们无需为 $G^{(k)}$ (当 k>1 时) 单独存储一套顶点特征向量。它的顶点特征向量就是它前一个图 $G^{(k-1)}$ 的边特征向量。

以下是计算第 $l+1$ 层新特征的精确流程：

***第一步：并行计算所有“更新增量”***

对于***每一个***图 $G^{(k)}$ (从 k=1 到最大阶 K)，我们基于第 $l$ 层的旧特征，计算出用于更新的“增量”或“原料”。

1.  ***计算“卷积更新增量” $u^{(k,l)}$***:
    -   对每个图 $G^{(k)}$ 中的每一个顶点 `α`，使用计算其卷积更新量 $u_{\alpha}^{(k,l)} = \phi_{u}\left(N_{m}^{-\alpha_{k}}\sum_{\beta\in\mathcal{N}^{(k)}(\alpha)}w_{\alpha\beta}^{k}m_{c,\alpha\beta}^{(k,l)}\right)$。
    -   这个计算会利用到 $G^{(k)}$ 的顶点特征 $v^{(k,l)}$ (也就是 $G^{(k-1)}$ 的边特征 $e^{(k-1,l)}$) 和 $G^{(k)}$ 的边特征 $e^{(k,l)}$。

2.  ***计算“自身消息更新增量” $m_s^{(k,l)}$***:
    -   对每个图 $G^{(k)}$ 中的每一个边 `αβ`，使用计算其自身消息更新量 $m_{s,\alpha\beta}^{(k,l)} = \phi_{s}(v_{\alpha}^{(k,l)}, v_{\beta}^{(k,l)}, e_{\alpha\beta}^{(k,l)})$。
    -   这个计算同样利用 $G^{(k)}$ 的顶点和边特征。

***第二步：应用增量，生成新特征***

现在，我们使用上一步计算出的所有“增量”，来生成第 $l+1$ 层的新特征。

1.  ***先更新边特征 $e^{(k-1, l+1)}$ (对于 k>1)***
    - $ e_{\alpha\beta}^{(k-1,l+1)} \leftarrow e_{\alpha\beta}^{(k-1,l+1)} + \delta_{s}^{(k-1,l)}m_{s,\alpha\beta}^{(k-1,l)} $

2.  ***更新顶点特征 $v^{(k, l+1)}$ (对于 k>1)***
    -   顶点特征***不需要***单独计算。根据核心等价关系，它直接由更新后的边特征定义：
    
        $v_{\alpha}^{(k,l+1)} \equiv e_{\alpha\beta}^{(k-1, l+1)}$

### 2.6.6 平滑截断：保证势能面光滑

为了避免在截断半径处能量突变，所有消息在聚合前都乘以一个平滑预因子(prefactor) $w_{\alpha\beta}^{k}$。这是通过一个***平滑开关函数*** $s^{k}(r_{ij})$ 实现的：

$$ s^{k}(r_{ij}) = \begin{cases} \exp(-\exp(C(r_{ij}-r_{cs}^{k})/r_{cs}^{k})) & \text{if } 0 < r_{ij} \le r_{c}^{k}, \\ 0 & \text{if } r_{ij} > r_{c}^{k}, \end{cases} $$

其中 $r_{cs}$ 是可以手动调整到平滑因子。这个函数能在截断半径 $r_c^k$ 附近从1平滑、指数级地衰减到0。通常对第一阶图，取 C = 20，$ r_c^1 $ = 6.0 Å，$ r_{cs}^1 $ = 5.3 Å

对于不同阶的图，预因子定义如下：

* ***对于$G^{(1)}$ (键)***: $w_{\alpha\beta}^{1} = w_{ij}^{1} = s^{1}(r_{ij})$
* ***对于$G^{(2)}$ (角)***: $w_{\alpha\beta}^{2} = w_{(ij)(im)}^{2} = s^{2}(r_{ij}) \times s^{2}(r_{im})$
* ***对于$G^{(3)}$ (二面角)***: $w_{\alpha\beta}^{3} = w_{(ijm)(ijn)}^{3} = s^{3}(r_{ij}) \times s^{3}(r_{im}) \times s^{3}(r_{in})$

这种设计确保了所有相互作用（包括高阶相互作用）都在截断半径处平滑消失，这是获得稳定、物理真实的分子动力学模拟结果的必要条件。


### 2.6.7 $G^{(1)}$的特殊更新：对称化操作 `symm`

作为最基础的图，$G^{(1)}$（原子图）的顶点特征更新除了之前讨论的来自邻居的卷积消息外，还额外增加了两项，以引入更丰富的环境信息。这个特殊更新只作用于$G^{(1)}$：

$$ v_{i}^{(1,l+1)} \leftarrow v_{i}^{(1,l+1)} + \delta_{s,0}^{(1,l)}\phi_{s,0}(v_{i}^{(1,l)}) + \delta_{s,1}^{(1,l)}\phi_{s,1}(\tilde{v}_{i}^{(1,l)}) $$

* $\phi_{s,0}$ 是一个MLP，负责处理***自消息***（self-message），即只与原子自身当前特征相关的更新。
* $\phi_{s,1}$ 是另一个MLP，负责处理经过***对称化变换***后的中间特征 $\tilde{v}_{i}^{(1,l)}$。

这个中间特征 $\tilde{v}$ 是通过 `symm` 算子，将原子的顶点特征和边特征与一个辅助的径向矢量 $h_{ij}$ 结合后，再拼接（concat）起来的：

$$ \tilde{v}_{i}^{(1,l)} = \text{concat}\left(\text{symm}(v_{j}^{(1,l)}, h_{ij}), \text{symm}(e_{ij}^{(1,l)}, h_{ij})\right) $$

其中，辅助矢量 $h_{ij}$ 定义为：

$$ h_{ij} = \frac{s^{1}(r_{ij})}{(r_{ij})^{2}} \times (x_{ij}, y_{ij}, z_{ij}) $$

$s^{1}(r_{ij})$ 是之前讨论过的平滑开关函数。

> ***`symm` 操作详解：***

该操作是模型表达能力的一个关键。它的核心思想是：在保持***旋转不变性***的前提下，将一个本身就旋转不变的特征（论文中称为 $a_j$，例如由距离得到的边特征）和一个随坐标系旋转会发生变化的矢量（论文中称为 $b_j$，例如径向矢量 $h_{ij}$）的信息有效地结合起来，从而构建出能够描述环境***各向异性***（anisotropy）的、新的高阶不变特征。它的数学形式由以下三步构成：

1.  ***构造中间矩阵 $\Psi_{pq}$***:
    $$ \Psi_{pq} = \frac{1}{\sqrt{N_m}}\sum_{j \in \mathcal{N}(i)} w_{ij}^{1} a_{j,p} b_{j,q} $$
    通过对中心原子 $i$ 的所有在阶段半径内的邻居 $j$ 进行加权求和，将邻居的特征 $a_j$ 和矢量 $b_j$ 的分量结合成一个矩阵 $\Psi$。由于求和操作与邻居的顺序无关，这个矩阵对于邻居的***置换是不变的***。这里q是空间维度，如xyz轴，而p和s这是特征维度上的。

2.  ***矩阵分裂 (Split)***:
    $$ \Psi_{pq}^{<} = \text{split}_p(\Psi_{pq}) $$
    这一步从矩阵 $\Psi$ 中沿着维度 $p$ 提取出一部分，得到一个新的矩阵 $\Psi^<$.

3.  ***构造最终不变特征***:
    $$ \text{symm}(a_j, b_j) = \text{flatten}\left(\sum_q \Psi_{pq} \Psi_{sq}^{<}\right) $$
    通过矩阵乘积和扁平化（flatten）操作，最终得到一个固定长度的、同时满足***旋转不变性***和***置换不变性***的特征向量。这个操作使得模型能够区分例如四面体和八面体这样具有不同对称性的局部原子环境。

> $\sum_q \Psi_{pq} \Psi_{sq}^{<}$ 详解：

这一数学设计，可以实现在严格保证***旋转不变性 (rotational invariance)*** 的前提下，构建出具有高度***表达能力和区分度 (expressiveness and discriminative power)*** 的原子环境描述子。简单的旋转不变量（如仅使用距离）会丢失关键的角度和方向信息，无法区分复杂的局部几何构型。

***1. 中间张量 $\Psi_{pq}$：编码各向异性环境***

`symm`算子的第一步是构造一个中间张量 $\Psi_{pq}$，其定义为：
$$ \Psi_{pq} = \frac{1}{\sqrt{N_m}}\sum_{j \in \mathcal{N}(i)} w_{ij}^{1} a_{j,p} b_{j,q} $$
这个张量的作用是编码中心原子局部环境的***各向异性***（anisotropy），即与方向相关的信息。它将一个旋转不变的输入特征向量 $a_j$ 与一个随坐标系旋转会发生变化的矢量 $b_j$ （如三维空间矢量）结合起来。

* ***维度分析***:
    * 令输入特征向量 $a_j$ 的维度为 $D_f$（这是一个模型超参数，例如64，对应顶点或者边的特征维度长度）。
    * 空间矢量 $b_j$ 的维度为 $D_s$（通常为3，对应x,y,z）。
    * 因此，中间张量 $\Psi$ 是一个维度为 ***$D_f \times D_s$*** 的矩阵。其中，行索引 `p` 的范围是 $[1, D_f]$，列索引 `q` 的范围是 $[1, D_s]$。

值得注意的是，$\Psi$ 本身并非旋转不变量，其数值会随坐标系的旋转而改变。

***2. 二次型操作：构造旋转不变量***

为了从非不变的张量 $\Psi$ 中导出严格的旋转不变量，模型采用了二次型（quadratic form）操作，即让 $\Psi$ 与其自身的一个变体 $\Psi^<$ 进行内积运算：
$$ M_{ps} = \sum_q \Psi_{pq} \Psi_{sq}^{<} = \Psi_{pq} \cdot (\Psi_{sq}^{<})^T $$
此操作的动机与实现原理如下：

* ***不变性的数学原理***: 该操作的核心在于对***空间维度索引 `q`*** 的求和。这个求和在数学上等价于矩阵 $\Psi$ 的行向量与矩阵 $(\Psi^<)^T$ 的行向量（即 $\Psi^<$ 的列向量）之间的点积。在三维空间中，当坐标系发生旋转时，矢量会通过一个旋转矩阵进行变换，这会导致 $\Psi$ 的列向量发生相应的线性变换。然而，两个矢量经过相同旋转变换后，它们的点积保持不变。因此，$\Psi$ 的任意两个行向量之间的点积在旋转操作下是不变的。这就从数学上保证了最终生成的矩阵 $M_{ps}$ 是一个严格的旋转不变量。

* ***$\Psi_{sq}^{<}$ 的维度***: $\Psi^<$ 是通过对 $\Psi$ 沿特征维度 `p` 进行分裂（split）或截断得到的。若截取前 $D_{f'}$ 维，则索引 `s` 的范围是 $[1, D_{f'}]$，其中 $D_{f'} \le D_f$。因此，$\Psi^<$ 是一个维度为 ***$D_{f'} \times D_s$*** 的矩阵。

***3. 丰富描述子：`p` 和 `s` 索引的作用***

如果只计算一个不变量（一个标量），描述子的信息量将非常有限。`symm` 操作通过构造一个完整的矩阵 $M_{ps}$ 来解决这个问题。

* ***高阶关联***: 矩阵 $M_{ps}$ 的维度为 ***$D_f \times D_{f'}$***。它的每一个元素 $M_{ps}$ 都代表了“由特征通道 `p` 感知的几何信息”与“由特征通道 `s` 感知的几何信息”之间的***关联强度***。这种关联是通过对空间维度 `q` 的内积来实现的。
* ***生成特征向量***: 通过计算整个矩阵的关联信息，并最终将其“压平”（flatten）成一个一维向量，模型便能生成一个包含了大量高阶几何关联的、信息极其丰富的特征向量。这个向量就像一个独特的“指纹”，能够有效地区分各种复杂的、具有不同对称性的局部原子环境，从而极大地提升了模型的表达能力。

### 2.6.8 效率优化
为了提升模型的效率，我们初步进行了一些优化工作，尽管这些工作并非系统性的。这些工作主要集中在特征维度压缩和“先线性后广播”策略上。

- ***特征维度压缩***：我们用 $n_{dim}$、$e_{dim}$ 和 $a_{dim}$ 分别表示节点、边和角度特征的维度，这些维度通常是依次递减的。为了提高效率，我们在形成消息之前，将节点和边的特征投影到 $a_{dim}$ 的大小： 
$$ \hat{n}_{i}^{l}=linear_{n_{dim}\rightarrow a_{dim}}(n_{i}^{l}) $$
$$ \hat{e}_{ij}^{l}=linear_{e_{dim}\rightarrow a_{dim}}(e_{ij}^{l}) $$
  - 然后我们可以改为计算 $\phi^{l}(\hat{n}_{i}^{l},\hat{e}_{ij}^{l},\hat{e}_{ik}^{l},a_{ijk}^{l})$。这种方法在很大程度上保持了准确性的同时，减少了计算量。

- ***先线性后广播***：在计算 $\phi^{l}(n_{i}^{l},n_{j}^{l},e_{ij}^{l})$ 或 $\phi^{l}(n_{i}^{l},e_{ij}^{l},e_{ik}^{l},a_{ijk}^{l})$ 时，典型的方法是首先根据各自的维度收集并拼接特征以获得输入，然后进行线性计算。然而，这种方法导致了大量的冗余计算。鉴于我们的线性层只包含一层，我们可以先对单个特征进行线性计算，然后利用自动广播来节省计算资源。


### 2.6.9 不变性原理

DPA-3模型被严格设计以遵守物理对称性，其最终的原子描述子 $v_{i}^{(1,L)}$ 满足所需的各种不变性。其原理如下：

* ***平移和旋转不变性 (Translational and Rotational Invariance)***:
    模型之所以满足这两种不变性，是因为它的所有输入信息——距离、角度、二面角——本身就是平移和旋转不变的。后续的所有MLP和消息传递操作（包括`symm`操作）都是在这些不变特征上进行的，因此其输出也必然是平......
    * ***初始化***: 所有初始特征（基于距离、角度、二面角）都是平移和旋转不变的。
    * ***层更新***: 由于消息 $m_c$ 和 $m_s$ 是由不变特征计算得出的，它们自身也是不变的。因此，逐层更新的过程会一直保持这种不变性。

* ***置换等变性 (Permutational Equivariance)***:
    对于同种元素的原子进行置换，模型必须给出一致的结果。这通过以下几点保证：
    * ***图结构***: 交换两个同种原子的标签，图的顶点集、边集以及连接关系保持不变。
    * ***初始化***: 原子的初始特征（one-hot）会随之交换，是等变的。边的初始特征集合虽然顺序可能改变，但集合本身不变。
    * ***层更新***: 核心的卷积操作 $\sum_{j \in \mathcal{N}(i)}$ 是对所有邻居信息的***求和***。求和与元素的顺序无关，因此无论邻居的顺序如何排列，得到的聚合信息都是相同的。这保证了更新过程与原子的置换顺序无关。
    * ***结论***: 由于 $G^{(1)}$ 的顶点特征的置换方式与原子的置换方式完全一致，并且整个更新过程保持了这种等变性，因此最终的原子描述子 $v_{i}^{(1,L)}$ 被证实是置换等变的，从而保证了总能量的置换不变性。


### 2.6.9 激活函数的鲁棒性

在训练 DPA-3 模型的过程中，某些归一化技术（如层归一化或批归一化）对准确性和效率都有负面影响。因此，模型架构中没有包含显式的归一化。另一方面，DPA-3采用了 SiLU 激活函数，其在准确性方面优于 tanh；

然而，它是一个无界激活函数。作者观察到，在训练过程中，特别是在较深的网络中，存在数值不稳定的趋势，导致数值爆炸性累积。因此，作者将 SiLU 替换为一个称之为 SiLUT（阈值 SiLU）的新激活函数，定义如下：
$$ SiLUT(x)=\begin{cases} SiLU(x) & \text{if } x \le t, \\ tanh(a\cdot(x-t))+b & \text{if } x > t, \end{cases} $$
其中 t 代表阈值，指示何时从 SiLU 过渡到 tanh。常数 a 和 b 根据 t 确定，以确保激活函数在阈值处的一阶和二阶连续性。作者发现，这种激活函数设计在大多数情况下能保持 SiLU 的准确性，同时提供更稳定的训练，尤其是在没有显式归一化的情况下。


### 2.6.10 训练
对于训练，模型中所有可训练参数 w 采用了 Adam 随机梯度下降法，以最小化损失函数：
$$ \mathcal{L}_{w}(E^{w},\mathcal{F}^{w})=\frac{1}{|\mathcal{B}|}\sum_{t\in\mathcal{B}}(p_{\epsilon}|E_{t}-E_{t}^{w}|^{2}+p_{f}|\mathcal{F}_{t}-\mathcal{F}_{t}^{w}|^{2}) $$
这里 $\mathcal{B}$ 代表一个小批量（minibatch），$|\mathcal{B}|$ 是批量大小，t 表示训练样本的索引。$E^{w}$ 和 $\mathcal{F}^{w}$ 表示模型输出，而 $E, \mathcal{F}$ 是相应的 DFT 结果。另外还采用了一个调度器来在训练过程中调整前置因子 $p_{\epsilon}$ 和 $p_{f}$，以便在能量和力的标签之间取得更好的平衡。如果有维里（virial）误差，也可以将其添加到损失函数中进行训练，此处省略。


# 3. Train DPA3 model from scratch

We will first train a 6-layer DPA3 model from scratch on the example water system.

In [1]:
!cd water_scratch && cat input_torch.json

{
    "model": {
        "type_map": [
            "O",
            "H"
        ],
        "descriptor": {
            "type": "dpa3",
            "repflow": {
                "n_dim": 128,
                "e_dim": 64,
                "a_dim": 32,
                "nlayers": 6,
                "e_rcut": 6.0,
                "e_rcut_smth": 5.3,
                "e_sel": 120,
                "a_rcut": 4.0,
                "a_rcut_smth": 3.5,
                "a_sel": 30,
                "axis_neuron": 4,
                "fix_stat_std": 0.3,
                "a_compress_rate": 1,
                "a_compress_e_rate": 2,
                "a_compress_use_split": true,
                "update_angle": true,
                "smooth_edge_update": true,
                "edge_init_use_dist": true,
                "use_exp_switch": true,
                "update_style": "res_residual",
                "update_residual": 0.1,
                "update_residual_init": "const"
            },
            "acti

## 3.1 Detailed description of parameters

参数具体介绍可以在该网页中查询：[DeePMD-kit Doc](https://docs.deepmodeling.com/projects/deepmd/en/v3.1.0rc0/train/train-input.html#argument:model[standard]/descriptor[dpa3]/repflow)。这里只给出一个参考和大致解释。

### 3.1.1 repflow

***n_dim:***

指定节点表示的维度（即特征向量长度），在论文中记为 $n_i^l$。该向量表示与每个原子相关的特征。

***e_dim:***

指定边表示的维度，在论文中记为 $e_{ij}^l$。该向量表示一对相邻原子（键）的特征。

***a_dim:***

指定角表示的维度，在论文中记为 $a_{ijk}^l$。该向量表示由三个原子组成的角的特征。

***nlayers:***

决定 repflow 层的数量，在论文中记为 $L$。这是图神经网络的深度，控制消息传递和特征更新步骤重复的次数。更多的层可以让模型学习更大范围的原子环境。

***e_rcut:***

边相互作用的截断半径，在论文中记为 $r_c^e$。它定义了两个原子被视为邻居以进行节点和边特征更新的最大距离。通常这里使用较大的截断半径以捕获更多环境信息。

***e_rcut_smth:***

平滑截断（切换）函数开始削弱边相互作用强度的距离。在论文中记为 $r_{cs}^e$。在此半径处相互作用从完全强度平滑过渡到在 `e_rcut` 处为零。

***e_sel:***

设置每个原子用于边计算的最大邻居数。由于实际邻居数变化，该参数通过零填充创建固定大小的张量，这对于 GPU 高效计算是必要的。"auto" 选项则使用最大的近邻原子数，"auto:factor" 则在 auto 的基础上乘以 factor，最终调整为4的倍数。

***a_rcut:***

角相互作用的截断半径，在论文中记为 $r_c^a$。它定义了涉及三体角信息更新的邻域。由于三体计算开销较大，通常比 `e_rcut` 小。

***a_rcut_smth:***

平滑截断（切换）函数开始削弱角相互作用的距离。与 `e_rcut_smth` 类似，但应用于 `a_rcut` 半径。

***a_sel:***

设置在 `a_rcut` 半径内用于角计算的最大邻居数。与 `e_sel` 类似，用于创建固定大小的张量以便高效计算。

***a_compress_rate:***

控制“特征维度压缩”优化。为提高角更新的效率，节点和边特征可投影（压缩）到更小的维度。非零整数 c 会将节点和边特征分别压缩到 `a_dim / c` 和 `a_dim / (2*c)`，再用于角消息计算。

***a_compress_e_rate:***

对角消息中的边特征压缩提供更细致的控制。启用压缩时，边特征维度被压缩为 `(a_dim * a_compress_e_rate) / (2 * a_compress_rate)`。

***a_compress_use_split:***

特征维度压缩的另一种方法。若设为 `True`，则直接取原始特征向量的子向量（split），而不是用线性层投影到更小维度。

***axis_neuron:***

定义对称化操作中子矩阵的维度。在论文中，控制 $\Psi_{sq}^{<}$ 的大小，决定保留和用于更新的对称张量分量数。

***fix_stat_std:***

数据归一化参数。论文指出未使用标准归一化层。该参数允许用用户自定义的固定标准差对输入特征进行简单归一化，而不是从训练数据计算，有助于提升训练稳定性。

***skip_stat:***

向后兼容的废弃参数，会强制 `fix_stat_std` 为特定值（0.3）。应直接使用 `fix_stat_std`。

***update_angle:***

布尔开关，控制是否更新角表示（$a_{ijk}^l$）。若为 `False`，模型不进行三体消息传递更新，等效于忽略显式角信息（仅初始化）。

***update_style:***

决定残差更新的数学形式。论文描述了一种残差连接：新表示为旧表示加上更新项。'res_residual' 为每个更新分量引入可学习权重。'res_avg' 和 'res_incr' 是更简单的替代残差形式。

***update_residual:***

当 `update_style` 为 'res_residual' 时，定义可学习残差权重向量（如 $r_{n,1}^{l}$）的初始标准差。论文建议用较小的初始值（如 0.1）。

***update_residual_init:***

指定可学习残差权重的初始化方法。可用正态分布（'norm'）或常数值（'const'）初始化。

***optim_update:***

布尔开关，启用“先线性后广播”效率优化。启用后，先对单个特征向量做线性计算再组合，避免对大拼接向量的冗余计算，加快训练。

***edge_init_use_dist:***

布尔开关，改变边特征的初始化方式。默认用距离的倒数（$1/r$）初始化。设为 `True` 时用直接距离（$r$）初始化。

***use_exp_switch:***

布尔开关，选择平滑截断函数类型。原通常使用多项式函数，DPA-3 论文引入指数型，可在两者间选择。推荐 rcut_smth = 5.3 for rcut = 6.0, and 3.5 for rcut = 4.0.

***use_dynamic_sel:***

布尔开关，启用更节省内存的邻居处理方式。不再将所有邻居列表填充到固定大小（`e_sel`），而是对每个原子用实际邻居数。适用于邻居数变化大的体系。

***sel_reduce_factor:***

仅在 `use_dynamic_sel` 为 `True` 时使用的归一化因子。动态选择允许将 `e_sel` 设为很大（最坏情况），该因子将归一化常数缩放到更合理范围，防止数值问题。


> 一些论文中不太涉及的参数

***n_multi_edge_message:***

更新节点特征时边消息的“头数”。类似于多头注意力机制，使模型能以多种方式同时处理邻居信息并组合结果，捕获更复杂的关系。（几乎不用）

***smooth_edge_update:***

控制边更新的平滑性。设为 `True` 时，当邻居超出角截断 `a_rcut` 时，角消息项用零向量填充，可保证这种平滑填充。否则使用自身填充。在使用 `use_dynamic_sel` 时该项必须为 `True`。

***use_dynamic_sel*** 

使用动态近邻原子index，会有一些额外的成本，但是会更光滑，相当于只要在rcut范围内的邻居原子，都会考虑会导致每个张量大小不一样，训练速度会一定程度上降低。

NOTE：当前 DeePMD 中固定线图为 2 阶，通过线图来更新边，角度信息。

### 3.1.2 Others

***concat_output_tebd:***

布尔值开关，决定是否在描述符（descriptor）的最终输出向量上拼接（concatenate）上中心原子的类型嵌入（type embedding）向量。类型嵌入是一个代表原子元素种类（如C、O、H）的特征向量。开启此选项会将这个明确的原子类型信息附加到描述符后面，一起送入后续的拟合网络，这可能有助于模型更好地区分不同种类原子的能量贡献。

***activation_function:***

指定在模型（尤其是在初始的嵌入网络中）使用的激活函数。激活函数为神经网络引入非线性，使其能够学习复杂的函数关系。DPA-3的论文中特别提到，他们发现SiLU激活函数比tanh等更精确，但为了解决其在深度网络中可能出现的数值不稳定问题，他们设计了一种名为thSiLU（或文档中的SiLUT）的带阈值的激活函数，它结合了SiLU的高精度和tanh的稳定性。

***precision:***

设定模型参数的计算精度。这在计算速度和数值稳定性之间做了一个权衡。
* `float32` (单精度) 是速度和精度的标准平衡点。
* `float64` (双精度) 提供了最高的数值稳定性，但会显著增加内存消耗和计算时间。
* `float16` (半精度) 速度最快，内存占用最小，但更容易出现数值溢出或下溢的问题。
* 对于 DPA3 模型，默认采用 fp32，其他模型默认 fp64。

***exclude_types:***

一个用于物理剪枝和计算优化的参数。它允许用户手动指定哪些原子类型对之间不存在相互作用。例如，在一个水和石墨烯的体系中，如果你只关心水和碳的相互作用，而不关心水分子内部的O-H作用，就可以在这里排除 [0, 1]（假设0是氧，1是氢）。程序在计算时会跳过这些被排除的原子对，从而节省计算资源。

***env_protection:***

一个用于防止计算中出现“除以零”错误的数值保护参数。在计算环境矩阵时，会涉及到类似 1/r 的项，其中 r 是原子间距离。在某些情况下（例如，由于填充(padding)导致两个邻居原子被放置在完全相同的位置），r 可能为零。这个参数会在分母上加上一个极小的值（epsilon），例如 1/(r + epsilon)，从而避免程序因除以零而崩溃。

***trainable:***

布尔值开关，决定描述符部分的网络参数是否在训练过程中被更新。
* `True` (默认)：描述符和拟合网络都参与训练。
* `False`：描述符部分的参数被“冻结”，只有后续的拟合网络参数会被训练。这在迁移学习中非常有用，例如，可以使用一个在大型通用数据集上预训练好的、固定的描述符，然后只在一个新的、特定的小数据集上训练拟合网络。

***seed:***

用于初始化模型网络权重的随机数种子。在神经网络中，权重通常是随机初始化的。设置一个固定的种子可以保证每次重新开始训练时，模型的初始权重都是完全一样的，这使得训练过程可以被精确复现，便于调试和比较不同超参数的效果。

***use_econf_tebd:***

布尔值开关，决定是否使用基于电子构型 (electronic configuration) 的类型嵌入。这是一种更精细的原子类型表示方法。标准的类型嵌入只区分元素（例如“碳”），而这个选项可能会使用更详细的信息（例如碳的1s²2s²2p²电子构型）来生成嵌入向量，为模型提供更丰富的物理化学信息。

***use_tebd_bias:***

布尔值开关，决定在类型嵌入层（通常是一个线性层）中是否使用偏置项（bias）。在线性变换 `y = Wx + b` 中，这个参数控制是否学习 `b`。这是神经网络层的一个标准选项。

***use_loc_mapping:***

布尔值开关，决定是否在训练或非并行推理时使用局部原子索引映射。在处理一个批次（batch）的数据时，原子有其在整个系统中的全局索引。启用此选项后，程序会为该批次内的原子创建一个从0开始的局部索引，并在计算邻居列表时使用这个局部索引。这可以简化索引管理，尤其是在并行计算环境下，可能会带来效率提升。

***resnet_dt:***

这个参数应用于拟合网络 (fitting_net) 而非描述符。它控制在拟合网络的残差连接（ResNet/skip connection）中是否使用一个类似于“时间步长”(Timestep)的因子。标准的残差连接是 `y = x + F(x)`，启用此选项后可能变为 `y = x + dt * F(x)`，其中 `dt` 是一个可学习的或固定的缩放因子。这种技巧有时可以改善深度网络的训练动态和性能。

***start_pref_e / limit_pref_e:***

设定训练起始和结束时，能量损失 (energy loss) 在总损失函数中所占的权重。DPA-3的损失函数是能量、力和 virial 损失的加权和：$\mathcal{L} = p_e|E|^2 + p_f|F|^2 + ...$。`start_pref_e` 就是初始的 $p_e$，$limit_pref_e$ 则是训练到无穷步时的 $p_e$。训练过程中，权重会从起始值平滑地过渡到结束值。

***start_pref_f / limit_pref_f:***

设定训练起始和结束时，力损失 (force loss) 在总损失函数中所占的权重($p_f$)。通常，在训练初期会给力设置一个非常大的权重（`start_pref_f` 较大），让模型先学会正确的相互作用力（即势能面的梯度），之后再慢慢降低力的权重，转而更关注能量的绝对值。

***start_pref_v / limit_pref_v:***

设定训练起始和结束时，维里/应力张量损失 (virial loss) 在总损失函数中所占的权重($p_v$)。这只在需要模拟有恒定压力（NPT系综）或需要精确计算应力的周期性体系中才重要。其权重变化逻辑与能量和力类似。当两者均设置为0时则不使用，默认均为0。

***batch_size:***

设定每次训练迭代中同时处理的原子构型（样本）数量。
* `int` 或 `list[int]`：为每个（或所有）系统指定固定的批次大小。
* `"auto:N"`：一种方便的自动模式。程序会自动调整批次大小，使得 `(批次大小 × 系统原子数)` 大约等于 `N`。这是一种很好的内存管理策略，可以保证GPU的计算负载相对稳定，同时避免因系统大小不一而导致的内存溢出。
* 其他模式如 `"mixed"`, `"max"`, `"filter"` 提供了更高级的批次数据处理和筛选策略。

## 3.2 Train

For demonstration simplicity, ***we will use only 200 training steps in this example***, note that optimal performance typically requires around ***1 million steps or more*** in real-world scenarios.

In [ ]:
! cd water_scratch && dp --pt train input_torch.json

# 4. Finetune from the pretrained DPA3 model

## 4.1 设置模型结构

Next, we provide DPA3 models pretrained on [***OpenLAM-v1 datasets***](https://www.aissquare.com/datasets/detail?pageType=datasets&name=OpenLAM-TrainingSet-v1&id=308), including application-oriented datasets such as Alloy, Semiconductor, and Drug-like molecules, as well as open source DFT datasets like OMat, MPtraj, and OC20.

This directory contains the following main files:
1. ***DPA-3.1-3M.pt*** : multi-task pretrained DPA3-L16 model on OpenLAM-v1 datasets, downloaded from [AIS-Square](https://www.aissquare.com/models/detail?pageType=models&name=DPA-3.1-3M&id=343).
2. ***input_pretrain_3.1_3M.json*** : the corresponding pretrained input for ***DPA-3.1-3M.pt***.


微调模型的时候，目标的微调模型和原预训练的模型的结构必须完全相同，即 'input.json' 中的 'model' 必须一样。

In [ ]:
import json

# Load pretrained model input
pretrained_input = json.load(open('./water_finetune/input_pretrain_3.1_3M.json', 'r'))

# Load example input
example_input = json.load(open('./water_finetune/input_torch.json', 'r'))

# Get pretrained model configuration
dpa3_descriptor = pretrained_input['model']['shared_dict']['dpa3_descriptor']
fitting = pretrained_input['model']['shared_dict']['shared_fitting']

# Do model configuration replacement
example_input['model']['descriptor'] = dpa3_descriptor
example_input['model']['fitting_net'] = fitting

# (Optional) set a smaller start learning rate
example_input['learning_rate']['start_lr'] = 1e-4

# Save finetune input
json.dump(example_input, open('./water_finetune/input_finetune.json', 'w'), indent=4)

In [ ]:
! cd water_finetune && cat input_finetune.json

Then, note that the pretrained model is pretrained in a multi-task manner, featuring a unified backbone (referred to as the unified descriptor) and several fitting nets for different datasets. For detailed information, refer to the [DPA2 paper](https://www.nature.com/articles/s41524-024-01493-2).

![alt](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/150/b46229d6746e4358be3181363a158efb/94935036-1adb-4562-ae45-b7839e8666ae.png)

Before finetuning, we need to ***select a specific fitting net*** from the model to make predictions. To list the available fitting nets (model-branch) in this pretrained model, use the following command:

In [ ]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt show DPA-3.1-3M.pt model-branch

This lists all the available fitting nets. Then you need to select a fitting net that closely matches your system, here we take `MP_traj_v024_alldata_mixu`, which stands for branch of MPtraj dataset for example.

In [ ]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt train input_finetune.json --skip-neighbor-stat --finetune DPA-3.1-3M.pt --model-branch MP_traj_v024_alldata_mixu

Then finally we have a finetuned DPA3 model.

## 4. Test models

After completing training/finetuning, we typically evaluate model performance by examining accuracy across different material systems.

First we can compare the loss curves (lcurve.out) during training/finetuning:

In [ ]:
!head /deepmd-kit/examples/water/dpa3/lcurve.out

In [ ]:
!head /deepmd-kit/examples/water/dpa3_finetune/lcurve.out

Then we will typically run inference directly on target systems using the `dp test` command to measure prediction errors, which provides more precise accuracy evaluation:

In [ ]:
!dp test --help

In [ ]:
!dp --pt test -m /deepmd-kit/examples/water/dpa3/model.ckpt.pt -s /deepmd-kit/examples/water/data/data_3

In [ ]:
!dp --pt test -m /deepmd-kit/examples/water/dpa3_finetune/model.ckpt.pt -s /deepmd-kit/examples/water/data/data_3

We can see that the fine-tuned model demonstrates significantly better accuracy (benefiting from the pre-trained model).

## 5. Python interface for MD

Finally, we demonstrate how to use trained models for ***practical simulations in Python via the ASE interface***. 

In [ ]:
!pip install ase

We need first freeze out checkout file into one `pth` frozen model:

In [ ]:
!cd /deepmd-kit/examples/water/dpa3_finetune && dp --pt freeze -o dpa3_water_finetune.pth

We can also directly freeze the pretrained model without further finetuning to perform MD: 

```
dp --pt freeze -c DPA-3.1-3M.pt -o dpa3_water_branch.pth --model-branch H2O_H2O_PD

```



Then we can run the following Python code for prediction or optimization:

In [ ]:
%%writefile ase_md.py
from ase import Atoms
from deepmd.calculator import DP

## Compute potential energy
dp = DP("/deepmd-kit/examples/water/dpa3_finetune/dpa3_water_finetune.pth")
water = Atoms('H2O', positions=[(0.7601, 1.9270, 1), (1.9575, 1, 1), (1., 1., 1.)], cell=[100, 100, 100])
water.calc = dp
print(water.get_potential_energy())
print(water.get_forces())

## Run BFGS structure optimization
from ase.optimize import BFGS
dyn = BFGS(water)
dyn.run(fmax=1e-3)
print(water.get_positions())

In [ ]:
!python ase_md.py

### Ref
1. [DPA3 paper](https://arxiv.org/abs/2506.01686)
2. [LAMBench paper](https://arxiv.org/abs/2504.19578)
2. [DPA2 paper](https://www.nature.com/articles/s41524-024-01278-7)
2. [DPA2 finetune tutorial](https://www.aissquare.com/models/detail?pageType=models&name=DPA-2.3.1-v3.0.0rc0&id=287)
3. [AIS-Square](https://www.aissquare.com)
4. [OpenLAM Initiative](https://www.aissquare.com/openlam)
5. [LAMBench Website](https://www.aissquare.com/openlam?tab=Benchmark)